# <center> PROCESAMIENTO DIGITAL DE SEÑALES DE AUDIO</center>
## <center> Transformada Q Constante</center>      

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile

import IPython.display as ipd

**NOTA:** *Las siguientes celdas instalan e importan la biblioteca [librosa](https://librosa.org/) para procesamiento de audio y música en python.*

In [ ]:
!pip install librosa

In [ ]:
import librosa
import librosa.display

In [ ]:
librosa.__version__

### Descripción

Este ejercicio busca estudiar una **representación tiempo-frecuencia multi-resolución** basada en la **Transformada Q constante (CQT)**. 

La tarea planteada consiste en estudiar el **banco de filtros Q constante** implementado en la biblioteca [librosa](https://librosa.org/), analizando sus parámetros y las características de los filtros. Luego se calcula el espectrograma basado en la STFT y el espectrograma basado la CQT para una señal de ejemplo y se comparan ambas representaciones tiempo-frecuencia.

### Cómo correr el notebook
Se puede bajar y correr el notebook de forma local en una computadora.

O también se puede correr en Google Colab usando el siguiente enlace. 

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mrocamora/audio-dsp/blob/main/notebooks/audioDSP-CQT_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

### Obtener un archivo

La siguiente celda descarga un archivo de audio de ejemplo.

In [ ]:
# read the audio file
y, sr = librosa.load(librosa.ex('trumpet'))

# play audio
ipd.Audio(y, rate=sr)

La siguiente celda grafica la forma de onda.

In [ ]:
# plot audio signal
plt.figure(figsize=(12,8))
ax1 = plt.subplot(2, 1, 1)
librosa.display.waveplot(y, sr=sr)
plt.title('audio waveform')
plt.tight_layout()

### Banco de filtros Q constante

En lo que sigue se diseña un banco de filtros Q constante usando [librosa](https://librosa.org/). Estudie los parámetros que recibe la función para el diseño del banco de filtros, analice el resultado obtenido y conteste las siguientes preguntas.

  1. ¿Cómo son los filtros en el dominio del tiempo (kernels temporales)? ¿Qué tipo de ventana se utiliza?
  2. ¿Cómo varía el soporte (largo efectivo) de los filtros en el domino del tiempo?
  3. ¿Cómo es la distribución de la frecuencia central de los filtros del banco?
  4. ¿Qué forma tiene la respuesta en frecuencia de cada filtro?
  5. ¿Cómo varía el ancho de banda de los filtros a medida de que crece la frecuencia?
  6. ¿En qué regiones de frecuencia el banco de filtros tiene más resolución en frecuencia?

A continuación se construye el banco de filtros y se representan gráficamente algunos de los filtros en el dominio del tiempo así como su frecuencia central y la magnitud de la respuesta en frecuencia de cada filtro.

In [ ]:
# compute and plot the constant Q filter bank

basis, lengths = librosa.filters.constant_q(sr=22050)

fig, ax = plt.subplots(nrows=2, figsize=(12, 8))
notes = librosa.midi_to_note(np.arange(24, 24 + len(basis)))

for i, (f, n) in enumerate(zip(basis, notes[:12])):
    f_scale = librosa.util.normalize(f) / 2
    ax[0].plot(i + f_scale.real)
    ax[0].plot(i + f_scale.imag, linestyle=':')

ax[0].set(yticks=np.arange(len(notes[:12])), yticklabels=notes[:12],
          ylabel='CQ filters',
          title='CQ filters (one octave, time domain)',
          xlabel='Time (samples at 22050 Hz)')
ax[0].legend(['Real', 'Imaginary'])

F = np.abs(np.fft.fftn(basis, axes=[-1]))

# Keep only the positive frequencies
F = F[:, :(1 + F.shape[1] // 2)]

librosa.display.specshow(F, x_axis='linear', y_axis='cqt_note', ax=ax[1])

ax[1].set(ylabel='CQ filters', title='CQ filter magnitudes (frequency domain)')
plt.tight_layout()

### Comparación de espectrograma con STFT  con CQT
En la siguiente celda se aplica un banco de filtros mel con las mismas características sobre el espectrograma de la señal de audio, para producir un espectrograma mel. Compare el espectrograma original y el espectrograma CQT. 

En particular considere lo siguiente.

  1. ¿Cómo es la distribución de las frecuencias de análisis?
  2. ¿Cómo es la resolución en frecuencias en cada caso?
  3. Modifique el parámetro `bins_per_octave` para cambiar la resolución en frecuencia.
  3. Modifique el parámetro `n_bins` para extender el rango de análisis en frecuencia.

In [ ]:
# Compute spectrogam from STFT
S = np.abs(librosa.stft(y))

# plot spectrogram
fig, ax = plt.subplots(figsize=(12, 8))
img = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                              y_axis='linear', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")
plt.tight_layout()

In [ ]:
# Compute spectrogam from CQT
C = np.abs(librosa.cqt(y, sr=sr, n_bins=84, bins_per_octave=12))

# plot CQT spectrogram
fig, ax = plt.subplots(figsize=(12, 8))
img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                               sr=sr, x_axis='time', y_axis='cqt_hz', ax=ax)
ax.set_title('Constant-Q power spectrum')
fig.colorbar(img, ax=ax, format="%+2.0f dB")
plt.tight_layout()